# Gesture Recognition - Model 9
#### CNN+LSTM
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly.

In [1]:
import numpy as np
import os
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. We will set the initial `batch_size` here. Note that we will experimentally set the batch size in such a way that you are able to use the GPU in full capacity. Keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

## Generator and preprocessing functions
This is one of the most important part of the code. The overall structure of the generator is broken down into modules. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. 

In [4]:
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
nb_frames = 30  # lenght of the video frames
nb_channel = 3  # number of channels in images 3 for color(RGB) and 1 for Gray

In [5]:
# Helper function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [7]:
# Helper function to initialize all the batch image data and labels
def initialize_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [8]:
def process_batch_images(source_path, folder_list, batch_num, batch_size, t,validation):
    
    batch_data,batch_labels = init_batch_data(batch_size)
    
    # For building an augumented batch data with affine transformation
    batch_data_aug,batch_labels_aug = initialize_batch_data(batch_size)
    
    # For building an augmented batch data with horizontal flip
    batch_data_flip,batch_labels_flip = initialize_batch_data(batch_size)
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, nb_frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generating a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cropping non symmetric frames
            if image.shape[0] != image.shape[1]:
                image=image[0:120,20:140]
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            #Normal data
            batch_data[folder,idx] = (resized)
            
            #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))
            
            # Data with horizontal flip
            batch_data_flip[folder,idx]= np.flip(resized,1)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
        if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
        elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                    
        else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
                  
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)

    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 
    batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

In [9]:
def generator(source_path, folder_list, batch_size, validation=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield process_batch_images(source_path, folder_list, batch, batch_size, t,validation)            
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield process_batch_images(source_path, folder_list, batch, batch_size, t,validation)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [10]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model 9

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, Conv2D, MaxPooling3D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.layers.recurrent import LSTM

#nb_filters = [8,16,32,64]
#nb_dense = [256, 128, 5]

nb_classes = 5

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2),
            activation='relu', padding='same'), input_shape=input_shape))
model.add(TimeDistributed(Conv2D(32, (3,3),
            kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(64, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(128, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Conv2D(256, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        
model.add(TimeDistributed(Conv2D(512, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3),
            padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

model.add(TimeDistributed(Flatten()))

model.add(Dropout(0.5))
model.add(LSTM(256, return_sequences=False, dropout=0.5))
model.add(Dense(nb_classes, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [12]:
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 60, 60, 32)    4736      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 58, 58, 32)    9248      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 29, 29, 32)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 29, 29, 64)    18496     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 29, 29, 64)    36928     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 14, 14, 64)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 14, 14, 128)   73856     
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [13]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [14]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [15]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [16]:
batch_size = 10
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 182s 3s/step - loss: 1.7697 - categorical_accuracy: 0.2123 - val_loss: 1.8554 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to model_init_2020-03-3003_22_32.582803/model-00001-1.76985-0.21217-1.85537-0.18000.h5
Epoch 2/20
67/67 [==============================] - 31s 462ms/step - loss: 1.7334 - categorical_accuracy: 0.2056 - val_loss: 1.7518 - val_categorical_accuracy: 0.2400

Epoch 00002: saving model to model_init_2020-03-3003_22_32.582803/model-00002-1.73336-0.20564-1.75175-0.24000.h5
Epoch 3/20
67/67 [==============================] - 32s 474ms/step - loss: 1.7174 - categorical_accuracy: 0.1891 - val_loss: 1.7255 - val_categorical_accuracy: 0.2200

Epoch 00003: saving model to model_init_2020-03-3003_22_32.582803/model-00003-1.71743-0.18905-1.72555-0.22000.h5
Epoch 4/20
67/67 [==============================] - 3

In [17]:
batch_size = 20
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 17s 491ms/step - loss: 1.7100 - categorical_accuracy: 0.2092 - val_loss: 1.7367 - val_categorical_accuracy: 0.1200

Epoch 00001: saving model to model_init_2020-03-3003_22_32.582803/model-00001-1.71001-0.20915-1.73665-0.12000.h5
Epoch 2/20
34/34 [==============================] - 15s 427ms/step - loss: 1.6389 - categorical_accuracy: 0.2549 - val_loss: 1.7681 - val_categorical_accuracy: 0.1600

Epoch 00002: saving model to model_init_2020-03-3003_22_32.582803/model-00002-1.63888-0.25490-1.76806-0.16000.h5
Epoch 3/20
34/34 [==============================] - 14s 424ms/step - loss: 1.6823 - categorical_accuracy: 0.1961 - val_loss: 1.7098 - val_categorical_accuracy: 0.2200

Epoch 00003: saving model to model_init_2020-03-3003_22_32.582803/model-00003-1.68230-0.19608-1.70984-0.22000.h5
Epoch 4/20
34/34 [==============================] - 14s 426ms/step - loss: 1.6393 - categorical_accuracy: 0.2059 - val_loss: 1.7585 - val_categorical_accurac

In [18]:
batch_size = 30
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
23/23 [==============================] - 12s 509ms/step - loss: 1.6987 - categorical_accuracy: 0.1787 - val_loss: 1.7482 - val_categorical_accuracy: 0.0750

Epoch 00001: saving model to model_init_2020-03-3003_22_32.582803/model-00001-1.69873-0.17874-1.74817-0.07500.h5
Epoch 2/30
23/23 [==============================] - 11s 471ms/step - loss: 1.6416 - categorical_accuracy: 0.2077 - val_loss: 1.6788 - val_categorical_accuracy: 0.2000

Epoch 00002: saving model to model_init_2020-03-3003_22_32.582803/model-00002-1.64163-0.20773-1.67881-0.20000.h5
Epoch 3/30
23/23 [==============================] - 10s 445ms/step - loss: 1.6862 - categorical_accuracy: 0.1884 - val_loss: 1.7627 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2020-03-3003_22_32.582803/model-00003-1.68616-0.18841-1.76270-0.25000.h5
Epoch 4/30
23/23 [==============================] - 10s 435ms/step - loss: 1.7193 - categorical_accuracy: 0.1691 - val_loss: 1.8097 - val_categorical_accurac

In [19]:
batch_size = 50
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
14/14 [==============================] - 7s 517ms/step - loss: 1.6721 - categorical_accuracy: 0.2222 - val_loss: 1.7300 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2020-03-3003_22_32.582803/model-00001-1.67214-0.22222-1.72996-0.20000.h5
Epoch 2/30
14/14 [==============================] - 6s 437ms/step - loss: 1.6525 - categorical_accuracy: 0.2778 - val_loss: 1.7545 - val_categorical_accuracy: 0.1500

Epoch 00002: saving model to model_init_2020-03-3003_22_32.582803/model-00002-1.65249-0.27778-1.75448-0.15000.h5
Epoch 3/30
14/14 [==============================] - 6s 436ms/step - loss: 1.7017 - categorical_accuracy: 0.1825 - val_loss: 1.8387 - val_categorical_accuracy: 0.1500

Epoch 00003: saving model to model_init_2020-03-3003_22_32.582803/model-00003-1.70165-0.18254-1.83869-0.15000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.4901161901614834e-11.
Epoch 4/30
14/14 [==============================] - 6s 436ms/step - loss: 1.67


Epoch 00027: saving model to model_init_2020-03-3003_22_32.582803/model-00027-1.65552-0.19841-1.79829-0.10000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 2.9103831839091474e-14.
Epoch 28/30
14/14 [==============================] - 6s 421ms/step - loss: 1.6796 - categorical_accuracy: 0.1587 - val_loss: 1.6222 - val_categorical_accuracy: 0.3000

Epoch 00028: saving model to model_init_2020-03-3003_22_32.582803/model-00028-1.67962-0.15873-1.62225-0.30000.h5
Epoch 29/30
14/14 [==============================] - 6s 421ms/step - loss: 1.6986 - categorical_accuracy: 0.2381 - val_loss: 1.7210 - val_categorical_accuracy: 0.2500

Epoch 00029: saving model to model_init_2020-03-3003_22_32.582803/model-00029-1.69861-0.23810-1.72103-0.25000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1.4551915919545737e-14.
Epoch 30/30
14/14 [==============================] - 6s 424ms/step - loss: 1.6942 - categorical_accuracy: 0.2302 - val_loss: 1.8895 - val_categorical_accuracy: 

In [20]:
batch_size = 90
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
8/8 [==============================] - 4s 557ms/step - loss: 1.6607 - categorical_accuracy: 0.1528 - val_loss: 1.5454 - val_categorical_accuracy: 0.3000

Epoch 00001: saving model to model_init_2020-03-3003_22_32.582803/model-00001-1.66073-0.15278-1.54536-0.30000.h5
Epoch 2/30
8/8 [==============================] - 4s 474ms/step - loss: 1.6502 - categorical_accuracy: 0.1944 - val_loss: 1.7053 - val_categorical_accuracy: 0.1000

Epoch 00002: saving model to model_init_2020-03-3003_22_32.582803/model-00002-1.65022-0.19444-1.70529-0.10000.h5
Epoch 3/30
8/8 [==============================] - 4s 470ms/step - loss: 1.6854 - categorical_accuracy: 0.1528 - val_loss: 1.5766 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model_init_2020-03-3003_22_32.582803/model-00003-1.68535-0.15278-1.57657-0.25000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 7.275957959772868e-15.
Epoch 4/30
8/8 [==============================] - 4s 456ms/step - loss: 1.6541 - cate

In [21]:
batch_size = 90
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
8/8 [==============================] - 4s 550ms/step - loss: 1.7128 - categorical_accuracy: 0.1806 - val_loss: 1.6699 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-03-3003_22_32.582803/model-00001-1.71278-0.18056-1.66987-0.25000.h5
Epoch 2/20
8/8 [==============================] - 4s 461ms/step - loss: 1.6379 - categorical_accuracy: 0.2917 - val_loss: 1.7838 - val_categorical_accuracy: 0.1500

Epoch 00002: saving model to model_init_2020-03-3003_22_32.582803/model-00002-1.63787-0.29167-1.78375-0.15000.h5
Epoch 3/20
8/8 [==============================] - 4s 464ms/step - loss: 1.6911 - categorical_accuracy: 0.1667 - val_loss: 1.6375 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2020-03-3003_22_32.582803/model-00003-1.69111-0.16667-1.63754-0.20000.h5
Epoch 4/20
8/8 [==============================] - 4s 454ms/step - loss: 1.6104 - categorical_accuracy: 0.2500 - val_loss: 1.9105 - val_categorical_accuracy: 0.1500

E